# Movie Recommender System

Basic recommender system using Python and pandas.

Movies will be suggested by similarity to other movies. To make the system more robust implement ability to find movies across users that both have rated well and the cross-reccommend movies that have not been viewed.

In [1]:
import numpy as np
import pandas as pd